In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import scipy.stats as stats
import statsmodels.api as sm
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf, adfuller, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf



# very good for ACF PACF : Interpreting ACF and PACF Plots for Time Series Forecasting
# https://towardsdatascience.com/interpreting-acf-and-pacf-plots-for-time-series-forecasting-af0d6db4061c

In [ ]:
df = pd.read_excel('2022_HiMCM_Data-B-co2.xlsx',sheet_name="origin(2)")
year = df['Year'].to_numpy()
ppm = df['PPM']
degree = df['Degree']


# df = pd.read_csv('shampoo.csv')
# year = df['Month'].to_numpy()
# # ppm = df['PPM']
# degree = df['Sales']

fig = go.Figure()
fig.add_traces(go.Scatter(x=year, y=degree.diff(2), mode = "lines", name = "Degree_diff(1)"))
# fig.add_traces(go.Scatter(x=year, y=degree, mode = "lines", name = "Degree"))

# fig.add_traces(go.Scatter(x=year, y=ppm.diff(1), mode = "lines", name = "ppm_diff(1)"))
# fig.add_traces(go.Scatter(x=year, y=degree, mode = "lines", name = "Degree"))

fig.update_layout( 
        title='Degree/Year (2-order differenced data)',
        title_x=0.5,
        # autosize=True,
        width=1000,
        height=500,
    )

fig.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller
result = adfuller(degree.diff(1).dropna())
print('ADF Statistic: %f' % result[0])
print('1d -> p-value: %f' % result[1])



from numpy import log

print("\ndegree ")

result = adfuller(degree.dropna())
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

result = adfuller(degree.diff(1).dropna())
print('ADF Statistic: %f' % result[0])
print('1d -> p-value: %f' % result[1])

print("\nppm ")

result = adfuller(ppm.dropna())
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

result = adfuller(ppm.diff(2).dropna())
print('ADF Statistic: %f' % result[0])
print('1d -> p-value: %f' % result[1])

In [ ]:
from pmdarima.arima.utils import ndiffs

y = degree
# y = ppm


## Adf Test
print(ndiffs(y, test='adf'))  # 2

# KPSS test
print(ndiffs(y, test='kpss'))  # 0

# PP test:
print(ndiffs(y, test='pp'))  # 2

In [ ]:
ddd = 2

fig = plt.figure(figsize=(12, 4))
ax1 = fig.add_subplot(111)
# ax2 = fig.add_subplot(312)
# ax3 = fig.add_subplot(313)


# plot_pacf(degree.diff(ddd).dropna(), method='ols', lags=(len(degree) - ddd - 1)/2 -1, ax=ax1)
# plot_pacf(degree.diff(ddd).dropna(),  method='ywm', lags=(len(degree) - ddd - 1)/2 -1, ax=ax1)

plot_acf(degree.diff(ddd).dropna(), lags=len(degree) - ddd - 1, ax=ax1)


plt.show()


# p = 2 or 1 or 6
# d = 1
# q = 0, 1



# 2,1,0
# 2,1,1
# 1,1,0
# 1,1,1
# 6,1,0
# 6,1,1

In [ ]:
# 2,1,1 ARIMA Model (p,d,q)
modelfit = sm.tsa.ARIMA(degree.dropna().to_numpy(), order=(2,2,1)).fit()  # 默认使用最小二乘generalized least squares(GLS)，‘mle’是极大似然估计
print(modelfit.summary())
modelfit.plot_diagnostics(figsize=(12,8))
plt.show()


# 2,1,0
# 2,1,1
# 1,1,0
# 1,1,1
# 6,1,0
# 6,1,1

In [ ]:
# Plot residual errors
residuals = pd.DataFrame(modelfit.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()
stats.normaltest(residuals)

In [ ]:
# # Actual vs Fitted
from statsmodels.graphics.tsaplots import plot_predict
# plot_predict(modelfit.get_prediction, dynamic=False)

modelfit_predict = sm.tsa.ARIMA(degree.dropna().to_numpy(), order=(2, 2, 1)).fit()
# print(mmodelfit_predict.summary())
forecast = modelfit_predict.predict(dynamic=False)

figp = go.Figure()
figp.add_traces(go.Scatter(x=year, y=degree, mode = "lines", name = "Actual"))
figp.add_traces(go.Scatter(x=year, y=forecast, mode = "lines", name = "Fitted"))

figp.update_layout( 
        title='Actual vs Fitted',
        title_x=0.5,
        autosize=False,
        width=1200,
        height=500,
    )

figp.show()


In [ ]:
# https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/

# from statsmodels.tsa.stattools import acf

# Create Training and Test -  split the data into train (80%) and test (20%).
train_len = int(len(degree)*0.50)
train = degree[:train_len]
test = degree[train_len:]


print(len(degree))
print(train_len)

history = [x for x in train]
predictions = list()


for t in range(len(test)):
    model = sm.tsa.ARIMA(history, order=(2,2,1))
    model_fit = model.fit()
    pred = model_fit.forecast(steps=1)
    yhat = pred[0]
    predictions.append(yhat)
    # Append test observation into overall record
    obs = test[t+train_len]
    history.append(obs)


print(predictions)


# Make as pandas series
fc_series = pd.Series(predictions, index=test.index)
# lower_series = pd.Series(conf[:, 0], index=test.index)
# upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12, 5), dpi=100)
plt.plot(train, label='training')
plt.plot(test, label='actual')
plt.plot(fc_series, label='forecast')
# plt.fill_between(lower_series.index, lower_series, upper_series,
#                  color='k', alpha=.15)
plt.title('Forecasts vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()


# Accuracy metrics
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    acf1 = acf(forecast-test)[1]                      # ACF1
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'acf1':acf1, 
            'corr':corr, 'minmax':minmax})

forecast_accuracy(np.array(predictions), test.values)

In [ ]:

# Build Model to train the values
# modelfit_future = sm.tsa.ARIMA(degree.dropna().to_numpy(), order=(2, 2, 4)).fit()
modelfit_future = sm.tsa.ARIMA(degree.dropna().to_numpy(), order=(2, 2, 1)).fit()

# 2,1,0
# 2,1,1
# 1,1,0
# 1,1,1
# 6,1,0
# 6,1,1

# print(modelfit_future.summary())


ystart = 1975
yend_true_data = 2021
yend = 2050

predict = modelfit_future.predict(start =0, end = yend_true_data - ystart, dynamic=False)

# print(type(predict))
print(len(predict))

# # Forecast year = 30
future_year = yend - yend_true_data + 1
forecast_year = modelfit_future.forecast(steps=future_year) 
# print(forecast_year)

predict = np.append(predict, forecast_year)

yearlong = np.linspace(ystart, yend-1,yend - ystart )

fig_future = go.Figure()
fig_future.add_traces(go.Scatter(x=yearlong[:yend_true_data-ystart+1], y=degree, name = "actual data"))
# fig_future.add_traces(go.Scatter(x=yearlong[yend_true_data-ystart+1:], y=forecast_year, name = "predict data"))
fig_future.add_traces(go.Scatter(x=yearlong, y=predict, name = "predict data"))


fig_future.update_layout(
        xaxis = dict(
            title = "Year"
        ),
        yaxis = dict(
            title = "Degree(Celcius)"
        ),
        title='Degree Increment with Years',
        title_x=0.5,
        autosize=True,
        width=1500,
        height=900,
    )
fig_future.show()




In [ ]:
import pmdarima as pm


model = pm.auto_arima(degree, start_p=1, start_q=1,
                      test='adf',       # use adftest (adf) to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)


# model = pm.auto_arima(degree)


print(model.summary())

model.plot_diagnostics(figsize=(7,5))
plt.show()